In [200]:
import math
import numpy as np
import pandas as pd

## ID3 algorithm

Using ID3 methond to build decision tree

In [643]:
class Node:
    def __init__(self, labels):
        self.children = []
        self.labels = labels #it is possible to get rid of these, bot for the simplicity of a solution I am leaving as is.
        self.splitting_feature = None
        self.splitting_values = []
        
    def set_splitting_feature(self, splitting_feature):    
        self.splitting_feature = splitting_feature
        
    def set_children_splitting_vals(self, splitting_values):
        self.splitting_values = splitting_values
    
    def set_children(self, children):
        self.children = children
    
    def get_children(self):
        return self.children
    
    def node_error_rate(self, x, y):
        _, best_label = max([(len(np.where(label == self.labels)[0]), label) for label in self.labels])
        correct = len(np.where(y == best_label)[0])
        acc = correct / len(y)
        return 1 - acc
    
    def predict(self, x):
        if len(self.children) == 0: # We are in leaf.
            return self.best_label
        idx = self.splitting_values.index(x[self.splitting_feature])
        return self.children[idx].predict(x)
    

In [644]:
class Leaf(Node):
    def __init__(self, labels):
        Node.__init__(self, labels)
        seq = [(len(np.where(label == labels)[0]), label) for label in labels]
        if len(seq) == 0:
            self.best_label = -1 #Temp quick solution for empty leafs
        else:
            _, self.best_label = max(seq)

In [645]:
class ID3Tree:
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self._build_tree()
        
    def _information_gain(self, arr, labels):
        def plogp(p): return -p * math.log2(p)
        def calc_entropy(labels):
            unique_labels = np.unique(labels)
            return sum([plogp(len(np.where(labels == label)[0]) / len(labels)) for label in unique_labels])
            
        unique_vals = np.unique(arr)
        information_gain = calc_entropy(labels)
        for v in unique_vals:
            feature_labels = labels[np.where(arr == v)[0]]
            entropy = calc_entropy(feature_labels)
            information_gain -= entropy * len(feature_labels) / len(labels)
        return information_gain
    
    def _build_tree(self):
        data = self.data
        labels = self.labels
        
        columns = list(np.linspace(0, data.shape[1] - 1, data.shape[1]).astype(np.int32))
        self.root = self._build_node(data, labels, columns)
    
    def _build_node(self, data, labels, columns):
        if len(np.unique(labels)) == 1 or len(columns) == 0:
            return Leaf(labels)
        gain, i = max([(self._information_gain(data[:, i], labels), i) for i in columns])
        children = []
        splitting_values = []
        columns.remove(i)
        for v in np.unique(data[:, i]):
            node = self._build_node(data[data[:, i] == v], labels[data[:, i] == v], columns)
            children.append(node)
            splitting_values.append(v)
        
        result = Node(labels)
        result.set_children(children)
        result.set_splitting_feature(i)
        result.set_children_splitting_vals(splitting_values)
        return result

    def predict(self, test_x):
        result = [self.root.predict(x) for x in test_x]
        return np.array(result)
    
    def predict_for_node(self, node, test_x):
        result = [node.predict(x) for x in test_x]
        return np.array(result)
        
    def accuracy(self, validate_x, validate_y):
        predictions = [self.root.predict(x) for x in validate_x]
        good_preds = sum([1 for i in range(len(predictions)) if predictions[i] == validate_y[i]])
        return good_preds / len(validate_y)

### REP method
Prunning bottom up, this way prunning is quicker in avarage case.

In [646]:
def reduced_error_pruning(tree, validate_x, validate_y):
    def tree_crawl(node, validate_x, validate_y):
        if len(node.children) == 0: # Leaf.
            return True
        
        indexes = np.array([node.splitting_values.index(x[node.splitting_feature]) for x in validate_x])
        children_prunned = False
        for i, child in enumerate(node.children):
            new_validate_x = validate_x[np.where(indexes == i)[0]]
            new_validate_y = validate_y[np.where(indexes == i)[0]]
            if tree_crawl(child, new_validate_x, new_validate_y):
                children_prunned = True
                if len(child.children) != 0:
                    node.children[i] = Leaf(node.children[i].labels) #prunning
        
        if children_prunned:
            current_err = node.node_error_rate(validate_x, validate_y)
            predictions = tree.predict_for_node(node, validate_x)
            good_preds = sum([1 for i in range(len(predictions)) if predictions[i] == validate_y[i]])
            acc = good_preds / len(validate_y)
            err = 1 - acc
            if current_err <= err: 
                return True #make prunning
        return False
    tree_crawl(tree.root, validate_x, validate_y)
    
def tree_size(tree):
    def tree_size(node):
        return sum([tree_size(c) for c in node.children]) + len(node.children)
    return tree_size(tree.root)

- simple example of tree prunning

In [647]:
y = np.array([1,1,-1,1,-1,-1,1,1,1,1,1,1,-1,-1,-1])
x = np.array([[1,1,2,2],[2,1,2,2],[1,1,1,2],[1,2,1,2],[2,3,2,2],
                [2,2,1,2],[3,2,2,1],[1,3,2,2],[3,3,2,1],[2,3,1,2],
                [3,1,1,1],[1,2,1,1],[2,3,1,1],[2,1,1,2],[2,2,1,1]])
t = ID3Tree(x, y)

In [648]:
print(tree_size(t))
print(t.accuracy(x,y))

10
0.8666666666666667


In [649]:
reduced_error_pruning(t, x, y)

In [650]:
print(tree_size(t))
print(t.accuracy(x,y))

8
0.8666666666666667


- prunning works for this case. In general case it should be done on validation set.